## Vùng để code Augment 

In [2]:
print("from torch.utils.data import Dataset,DataLoader||from sklearn.model_selection import train_test_split||from pytorch_lightning.loggers import TensorBoardLogger||from pytorch_lightning.callbacks import ModelCheckpoint||from torchmetrics.functional.classification import accuracy||from tokenizers import Tokenizer||from tokenizers.models import WordPiece||from tokenizers.trainers import WordPieceTrainer".replace("||","\n"))

from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics.functional.classification import accuracy
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer


In [ ]:
"from torch.utils.data import Dataset,DataLoader||from sklearn.model_selection import train_test_split||from pytorch_lightning.loggers import TensorBoardLogger||from pytorch_lightning.callbacks import ModelCheckpoint||from torchmetrics.functional.classification import accuracy||from tokenizers import Tokenizer||from tokenizers.models import WordPiece||from tokenizers.trainers import WordPieceTrainer"

In [ ]:
import re
from spacy.lang.vi import Vietnamese
import string

nlp = Vietnamese()


def text_normalize(
    text,
    remove_urls=True,
    remove_emails=True,
    remove_mentions=True,
    remove_hashtags=True,
    remove_numbers=False,
):
    """
    Normalize and clean Vietnamese text with additional regex steps:
    - remove/normalize URLs, emails, mentions, hashtags, phone numbers, currency symbols
    - remove non-printable characters, reduce repeated punctuation and elongated letters
    - optional removal of digits
    Returns cleaned string (tokens joined by space) after spaCy tokenization and stopword filtering.
    """
    try:
        text = str(text)
        # basic normalization
        text = text.strip().lower()

        # remove urls
        if remove_urls:
            text = re.sub(r"https?://\S+|www\.\S+", " ", text)
        # remove emails
        if remove_emails:
            text = re.sub(r"\S+@\S+", " ", text)
        # remove html tags
        text = re.sub(r"<.*?>", " ", text)
        # mentions (@username)
        if remove_mentions:
            text = re.sub(r"@\w+", " ", text)
        # hashtags: keep the word but drop the #
        if remove_hashtags:
            text = re.sub(r"#(\w+)", r"\0", text)
        # remove phone numbers (simple patterns)
        text = re.sub(r"\b-1\d{8,}\b|\b\d{9,}\b", " ", text)
        # replace common currency symbols with space
        text = re.sub(r"[\$€£¥₫]", " ", text)

        # remove punctuation (translate to space to avoid joining words)
        text = re.sub(r"[%s]" % re.escape(string.punctuation), " ", text)

        # remove non-printable characters
        text = "".join(ch for ch in text if ch.isprintable())

        # collapse repeated punctuation (e.g., '!!!' -> '!')
        text = re.sub(r"([!?.]){1,}", r"\1", text)
        # reduce elongated characters (>1 repeats -> 2 repeats) e.g. heyyyy -> heyy
        text = re.sub(r"(.)\0{2,}", r"\1\1", text)

        # optionally remove digits entirely
        if remove_numbers:
            text = re.sub(r"\d+", " ", text)

        # normalize whitespace
        text = re.sub(r"\s+", " ", text).strip()

        # tokenize with spaCy (Vietnamese) and filter stopwords/punct/space
        doc = nlp(text)
        cleaned_tokens = []
        for token in doc:
            # Keep tokens that are not stopwords/punctuation/space
            if not token.is_stop and not token.is_punct and not token.is_space:
                # filter out tokens that contain digits (optional already handled above)
                if not any(char.isdigit() for char in token.text):
                    cleaned_tokens.append(token.lower_)

        return " ".join(cleaned_tokens)

    except Exception as e:
        print("Error normalizing text:", e)
        return ""


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel, BPE, WordPiece, Unigram
from tokenizers.trainers import (
    WordLevelTrainer,
    BpeTrainer,
    WordPieceTrainer,
    UnigramTrainer,
)
from tokenizers.pre_tokenizers import Whitespace
import pandas as pd

encodings = ["utf-8", "utf-8-sig", "latin1", "ISO-8859-1", "cp1252"]

for enc in encodings:
    try:
        df = pd.read_csv("./Attachments/train.csv", encoding=enc)
        print(f"Read finish with: {enc}")
        print(df.head())
        break
    except UnicodeDecodeError:
        print(f"Cant read with: {enc}")

corpus = df["question_text"].apply(lambda x: str(x)).tolist()  # type: ignore

data_size = len(corpus)


def create_tokenizer(vocab_size: int = 31000, sequence_length: int = 64):
    # tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))  # type: ignore
    tokenizer = Tokenizer(Unigram()) # type: ignore
    tokenizer.pre_tokenizer = Whitespace()  # type: ignore
    tokenizer.enable_padding(pad_id=0, pad_token="[PAD]", length=sequence_length)
    tokenizer.enable_truncation(max_length=sequence_length)
    trainer = UnigramTrainer(
        vocab_size=vocab_size, special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]", "[MASK]"],
        unk_token="[UNK]",
    )
    # trainer = WordPieceTrainer(
    #     vocab_size=vocab_size,
    #     special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]", "[MASK]"],
    #     unk_token="[UNK]",
    # )
    tokenizer.train_from_iterator(corpus, trainer=trainer)
    tokenizer.save("./Attachments/tokenizer-uni.json")
    return tokenizer


## Biến đổi dữ liệu

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
from tokenizers import Tokenizer

encodings = ["utf-8", "utf-8-sig", "latin1", "ISO-8859-1", "cp1252"]

for enc in encodings:
    try:
        df = pd.read_csv("./Attachments/train.csv", encoding=enc).dropna()
        # df = pd.read_csv("../Attachments/NER_dataset_cleaned.csv", encoding=enc)
        print(f"Read finish with: {enc}")
        print(df.head())
        break
    except UnicodeDecodeError:
        print(f"Cant read with: {enc}")


for enc in encodings:
    try:
        df_test = pd.read_csv("./Attachments/test.csv", encoding=enc).dropna()
        # df = pd.read_csv("../Attachments/NER_dataset_cleaned.csv", encoding=enc)
        print(f"Read finish with: {enc}")
        print(df_test.head())
        break
    except UnicodeDecodeError:
        print(f"Cant read with: {enc}")

df_0 = df[df["target"] == 0].iloc[:80000]
df_1 = df[df["target"] == 1]
df = pd.concat([df_0, df_1])
df = df.sample(frac=1).reset_index(drop=True)


class CustomCollator(Dataset):
    def __init__(self, questions, target, tokenizer, is_training= True):
        self.questions = questions
        self.target = target
        self.tokenizer = tokenizer
        self.is_training = is_training

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):

        # áp dụng biến đổi encoding và normalization vào dữ liệu
        question = text_normalize(self.questions[idx])
        encoding = self.tokenizer.encode(question).ids
        """
        NOTE: tokenizer dùng cho huggingface AutoTokenizer.from_pretrained thì sẽ trả về một dict gồm các key như input_ids, attention_mask, token_type_ids,...
        nên khi dùng huggingface thì phần này sẽ là:
        encoding = self.tokenizer(question, padding='max_length', truncation=True, max_length=sequence_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].squeeze(),  # remove batch dimension
            'attention_mask': encoding['attention_mask'].squeeze(),
            ...
        }
        """

        if self.is_training:
            target_id = [0, 0]
            target_id[int(self.target[idx])] = 1
            return (
                torch.tensor(encoding, dtype=torch.long),
                torch.tensor(target_id, dtype=torch.float),
            )
        else:
            return torch.tensor(encoding, dtype=torch.long)


tokenizer = create_tokenizer()
# tokenizer = Tokenizer.from_file("./Attachments/tokenizer-wp.json")

X_train, X_val, y_train, y_val = train_test_split(
    df["question_text"].apply(lambda x: str(x)).tolist(),  # type: ignore
    df["target"].apply(lambda x: int(x)).tolist(),  # type: ignore
    test_size=0.2,
    random_state=42,
)

X_test = df_test["question_text"].apply(lambda x: str(x)).tolist()  # type: ignore

training_dataset = CustomCollator(X_train, y_train, tokenizer)  # type: ignore
validation_dataset = CustomCollator(X_val, y_val, tokenizer)  # type: ignore
testing_dataset = CustomCollator(X_test, y_val, tokenizer, is_training=False)  # type: ignore


training_loader = DataLoader(
    training_dataset, batch_size=128, shuffle=True, drop_last=True
)  # type: ignore

validation_loader = DataLoader(
    validation_dataset, batch_size=128, shuffle=False, drop_last=True
)  # type: ignore

testing_loader = DataLoader(
    testing_dataset, batch_size=128, shuffle=False, drop_last=False
)  # type: ignore


## Áp mô hình 
 - pytorch lightning thì sẽ có những hàm default thì chỉ cần gọi ra là xong 
 - metric nó có sẵn rồi với thư viện torchmetrics.functional.<bài toán> 
 - chủ yếu xoay quanh những hàm hữu ích sau: 
    1. forward
    2. training_step
    3. vaidation_step
    4. test_step (bước này phải gọi trainer.test)
    5. configure_optimizers (này thì fix cứng)
 - sử dụng self.log để hiển thị log 

In [ ]:
import pytorch_lightning as L
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics.functional.classification import accuracy
import torch
from torch import nn
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

class MyModel(L.LightningModule):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        # Model components
        """
        self.trans = nn.Transformer(
            d_model=input_dim,
            activation="gelu",
            batch_first=True,
            dropout=0.2,
            num_encoder_layers=4,
            num_decoder_layers=2,
            nhead=8,
            # dim_feedforward=64*256,
            # norm_first=True,
        )  # 32x64x64
        self.dense = nn.Linear(input_dim, output_dim)
        """
        
        # Set loss với acc 
        self.criterion = torch.nn.CrossEntropyLoss() 
        self.accuracy = accuracy

        # Define your model architecture here

    def forward(self, x):
        # Forward pass logits computation
        """
        transformer_out = self.trans.encoder(x)  # 32x64x64
        transformer_out = self.dense(transformer_out[:, 0, :])  # get [CLS] token  32x2
        return transformer_out
        """
        pass

    def test_val_step(self, batch, batch_idx):
        # Shared step for validation and testing
        """
        x, y = batch # NOTE: Phụ thuộc vào return từ CustomCollator 
        logits: torch.Tensor = self(x)  # 32x2
        loss = self.criterion(logits, y)
        acc = self.accuracy( 
            logits.argmax(dim=-1),
            y.argmax(dim=-1),
            num_classes=self.output_dim,
            task="multiclass",
        ) # type: ignore
        return loss, acc
        """
        pass

    def training_step(self, batch, batch_idx):
        """
        loss, acc = self.test_val_step(batch, batch_idx)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return { # training step phải có trả về {"loss": loss} 
            "loss": loss,
            "accuracy": acc,
        }
        """
        pass

    def validation_step(self, batch, batch_idx):
        # gọi lại hàm trên thôi 
        """
        loss, acc = self.test_val_step(batch, batch_idx)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        """
        pass
    
    def on_validation_epoch_end(self):
        all_logits = self.validation_results["logits"]
        all_labels = self.validation_results["labels"]

        ## NOTE: Plot confusion matrix to TensorBoard
        cm = confusion_matrix(
            all_labels,
            all_logits,
            labels=[0, 1, 2, 3, 4, 5, 6, 7],
        )
        fig, ax = plt.subplots(figsize=(6, 6))

        disp = ConfusionMatrixDisplay(
            confusion_matrix=cm
        )
        disp.plot(ax=ax)

        f1_micro = f1_score(
            self.validation_results["logits"],
            self.validation_results["labels"],
            average="micro",
        )

        self.logger.experiment.add_figure(
            "validation/confusion_matrix", fig, self.current_epoch
        )
        self.log("validation/f1_micro", f1_micro, prog_bar=True)


        self.validation_results["logits"].clear()
        self.validation_results["labels"].clear()
        self.validation_results["texts"].clear()
        print(f"F1 Micro: {f1_micro}")

    def configure_optimizers(self):  # type: ignore
        # Cái này thì sét cứng chỉ cần thay đổi Lr thôi 
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-5)
        return {
            "optimizer": optimizer,
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode="min", patience=3
            ),
            "monitor": "val_loss",
            "interval": "epoch",
            "frequency": 1,
        }


def train(model):
    model_name = "Text_Classifier_Model"
    logger = TensorBoardLogger("tb_logs", name=model_name)
    
    # Save checkpoint điểm có val_acc tốt nhất 
    checkpoint_callback = ModelCheckpoint(
        monitor="val_acc",
        dirpath=f"checkpoints/{model_name}",
        filename="{model_name}-{epoch:02d}-{val_loss:.2f}",
        save_top_k=3,
        mode="max",
    )
    trainer = L.Trainer(
        max_epochs=5,
        accelerator="auto",
        devices="auto",
        logger=logger,
        gradient_clip_val=1.0,
        callbacks=[checkpoint_callback],
    )
    
    # Train, test với val 
    trainer.fit(
        model, train_dataloaders=training_loader, val_dataloaders=validation_loader
    )
    trainer.test(model, dataloaders=testing_loader)


if __name__ == "__main__":
    model = MyModel(input_dim=64, output_dim=2)
    train(model)
